# **Chapter 8 - Dimensionality Reduction**
Many Machine Learning Problems Involve thousands or even hundreds of features to Train the Models which In turn gives birth to **the curse of dimensionality**(Explained Later in the Chapter) and as a solution to this Problem comes Dimensionality Reduction here are two of the advantages of using Dimensionality Reduction :-
- Speed up the Process of Training.
- Helpful in Data Visualization.

# The Curse of Dimensionality
- Sometimes Our Machine Learning Model gets confused with the Noise of the Dataset.
    - Most of the times due to high number of features.
- High Numbers of features in a dataset results in slow speed of Training.